In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import re

In [ ]:
df = pd.read_csv('drive/My Drive/nlp-telugu/inltk/dataset/telugu_wiki_links.csv')
unique_links = df['link'].values

In [ ]:
def readout_buffer(response):
    response.text = response.read()
    return response.text.decode('utf-8')

    
def get_data_from_url(url):
    try:
        r = urlopen(url)
        doc = readout_buffer(r)
    except Exception as e:
        # print(e)
        doc = ""
    return doc

In [ ]:
!pip install selectolax
from selectolax.parser import HTMLParser
def get_details(url):
    doc = get_data_from_url(url)
    try: 
        html_doc = HTMLParser(doc)
        t = '\n '.join(n.text() for n in html_doc.css("title"))
        a = '\n '.join(n.text() for n in html_doc.css("p"))
    except:
        t = ""
        a = ""
    return t, a

In [ ]:
from datetime import datetime
import multiprocessing as mp
import multiprocessing.dummy as mpd
import time

start = datetime.now()
cpu_cores = mp.cpu_count()
print('parallelising the task on {} cpu cores'.format(cpu_cores))

rows = []
count = 0
# divide pool
pool = mpd.Pool(processes=cpu_cores)
# iter over 
for row in pool.imap(get_details, unique_links):
    rows.append(row)    
    count = count + 1
    # print/save
    if not count%100:
        print('done for,', count)
    if not count%10000:
        df = pd.DataFrame(rows, columns = ['title', 'text'])
        df.to_parquet('telugu_wikipedia_dataset.parquet', index = None)
        print("Done for {} rows ---> {}".format(count, datetime.now() - start))
# close the pool
pool.close()
pool.join()